# Query the EFD

Craig Lage - 06-Aug-24

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd') # 'usdf_efd' at USDF

# Sample query

In [ ]:
start = Time("2025-01-25 14:00:00Z", scale='utc')
end = Time("2025-01-27 11:00:00Z", scale='utc')
#start = Time("2025-01-25 14:45:00Z", scale='utc')
#end = Time("2025-01-25 14:50:00Z", scale='utc')
vac = await client.select_time_series('lsst.sal.ATCamera.vacuum', ['*'], start, end)
len(temps)

In [ ]:
vac.columns

In [ ]:
vac

In [ ]:
vac['private_sndStamp'].plot(marker='x')
plt.ylabel("lsst.sal.ATCamera.vacuum.private_sndStamp")
#plt.title("2025-01-22T22:01:34 UTC")
#plt.title("2025-01-25T14:47:15 UTC")

In [ ]:
vac['private_sndStamp'].values - vac['private_sndStamp'].values[0]

In [ ]:
temps['temp10'].plot()
temps['temp1'].plot()

In [ ]:
start = Time("2025-01-22 20:00:00Z", scale='utc')
end = Time("2025-01-23 17:00:00Z", scale='utc')
vac = await client.select_time_series('lsst.sal.ATCamera.vacuum', ['*'], start, end)
len(vac)

In [ ]:
start = Time("2025-02-27 16:20:00Z", scale='utc')
end = Time("2025-02-27 16:26:00Z", scale='utc')
sal = await client.select_time_series('lsst.sal.Script.logevent_description', ['*'], start, end)
salIndex = sal['salIndex'].values[0]
print(salIndex)
msg = await client.select_time_series('lsst.sal.Script.logevent_logMessage', ['*'], start, end)
for i in range(len(msg)):
    if msg.iloc[i]['salIndex'] == salIndex:
        print(msg.iloc[i]['traceback'])

In [ ]:
sndStamps = vac["private_sndStamp"]
timestamps = sndStamps.values
print(len(timestamps))
deltaTs = []
for n in range(1, len(timestamps)):
    deltaTs.append(timestamps[n] - timestamps[n-1])
sig = np.std(deltaTs)
%matplotlib inline
plt.title(f"private_sndStamps deltas")
plt.plot(deltaTs, marker='x')
#plt.text(2.0, 2.2,f"DeltaT sigma = {sig:.2g}")
plt.xlabel("Time(sec)")
plt.ylabel("DeltaT between successive timestamps (sec)")


In [ ]:
rot['nasmyth1CalculatedAngle'].plot()

In [ ]:
az.columns

In [ ]:
az.head(3)

In [ ]:
az['azimuthPosition'].plot()

# Querying available topics

In [ ]:
await client.get_topics()

In [ ]:
topics = await client.get_topics()
for topic in topics:
    if 'Hexapod' in topic:
        print(topic)

In [ ]:
start = Time("2024-10-07T12:00:00", scale='utc')
end = Time("2024-10-07T15:00:00", scale='utc')
acc = await client.select_time_series("lsst.sal.MTM1M3.accelerometerData", \
                                            ['*'], \
                                             start, end)
gyro = await client.select_time_series("lsst.sal.MTM1M3.gyroData", \
                                            ['*'], \
                                             start, end)
print(len(acc), len(gyro))

In [ ]:
acc.head(2)

In [ ]:
acc.tail(2)

In [ ]:
start = Time("2024-10-07T18:40:00", scale='utc')
end = Time("2024-10-07T19:30:00", scale='utc')
acc = await client.select_time_series("lsst.sal.MTM1M3.accelerometerData", \
                                            ['*'], \
                                             start, end)

plt.title('MTM1M3 Accelerometers - 07-Oct-24', fontsize=18) 
for i in range(8):
    acc[f'accelerometer{i}'].plot(label=f'{i}')
plt.ylim(-0.5, 0.5)
plt.legend(loc='upper left')
plt.savefig("/home/cslage/DATA/MTM13_Accelerometers_07Oct24.png")

In [ ]:
start = Time("2024-10-07T18:40:00", scale='utc')
end = Time("2024-10-07T19:30:00", scale='utc')
gyro = await client.select_time_series("lsst.sal.MTM1M3.gyroData", \
                                            ['*'], \
                                             start, end)
acc = await client.select_time_series("lsst.sal.MTM1M3.accelerometerData", \
                                            ['*'], \
                                             start, end)
mtMount = await client.select_time_series("lsst.sal.MTMount.azimuth", \
                                            ['*'], \
                                             start, end)


In [ ]:
fig = plt.figure(figsize=(10,8))
plt.subplots_adjust(hspace=0.5)
plt.subplot(2,3,1)
plt.title('Azimuth position', fontsize=18)
mtMount['actualPosition'].plot()
plt.subplot(2,3,2)
plt.title('Azimuth velocity', fontsize=18)
mtMount['actualVelocity'].plot()
plt.ylim(0.20, -0.20)
plt.subplot(2,3,3)
plt.title('Azimuth acceleration', fontsize=18)
mtMount['actualAcceleration'].plot()
plt.ylim(-5.0, 5.0)
plt.subplot(2,3,5)
plt.title('MTM1M3 Gyros', fontsize=18) 
axes = ['X', 'Y', 'Z']
for axis in axes:
    gyro[f'angularVelocity{axis}'].plot(label=f'{axis}')
plt.legend(bbox_to_anchor=(-0.1, 0.8))
plt.subplot(2,3,6)
plt.title('MTM1M3 Accels', fontsize=18) 
for i in range(8):
    acc[f'accelerometer{i}'].plot(label=f'{i}')
plt.ylim(-0.2, 0.2)
plt.legend(bbox_to_anchor=(1.05, 0.9))

plt.savefig("/home/cslage/DATA/MTM13_Gyros_Accels_07Oct24.png")